In [1]:
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [3]:
documents = []
for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
     

In [4]:
print(documents[2])

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}


In [5]:
pip install sentence_transformers==2.7.0  # we need to install this librabry 

Note: you may need to restart the kernel to use updated packages.


In [6]:
from sentence_transformers import SentenceTransformer # https://sbert.net/

In [7]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-mpnet-base-v2") # ovaj model prebacuje text u vector

/usr/local/python/3.10.13/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
model.encode("This is simple text")

array([ 6.10516313e-03, -1.30663261e-01,  3.07155220e-04,  5.51674217e-02,
       -1.66511927e-02,  4.26352918e-02,  1.93099454e-02,  2.92037744e-02,
        1.70986913e-03, -1.47201940e-02,  5.38685471e-02, -1.06397439e-02,
        6.93790689e-02,  1.24955270e-02, -3.05773970e-03, -8.90212581e-02,
        5.57089336e-02,  2.85896752e-02,  4.05467860e-02,  1.06600216e-02,
        3.35059054e-02,  1.19440835e-02, -8.32875539e-03, -1.07954117e-02,
        3.56376767e-02, -1.03118131e-02, -1.17484853e-02, -1.12689100e-02,
       -1.13780852e-02, -3.85779180e-02, -2.28792224e-02,  1.46114146e-02,
        1.35770245e-02, -7.18140304e-02,  1.76964818e-06,  2.71628116e-04,
       -1.47544295e-02, -6.86573470e-03, -2.31716782e-02,  4.09029871e-02,
        4.49574582e-04,  5.23888953e-02, -4.98281457e-02,  3.15837376e-02,
       -3.12922057e-03,  5.92366457e-02,  4.33208533e-02,  7.16993436e-02,
       -1.94266643e-02,  6.28504455e-02, -1.78020727e-03, -5.42235486e-02,
       -1.21343853e-02, -

In [9]:
len(model.encode("This is  a simple sentence"))  # velicina vecotra nam je jako bitna ovaj model(all-mpnet-base-v2)nam daje tu velicinu768

768

In [10]:
# ovaj proces je jako spor !!!!!
# kreiramo  embeding na osnovu nekog polja iz json-aa, uzecemo text field iz json-a
operations = [] 

for doc in documents:
    doc["text-vector"] = model.encode(doc["text"]).tolist()  # kreiramo vecotr propery unutar naseg json-a  sa imenom text-vector"
    operations.append(doc)   # ovo predstavalja kopiju originalngo json-a sa novim poljem koje predstavalja nas embedings

In [11]:
print(operations[1])

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites', 'section': 'General course-related questions', 'question': 'Course - What are the prerequisites for this course?', 'course': 'data-engineering-zoomcamp', 'text-vector': [-0.041030403226614, 0.025834161788225174, -0.036801841109991074, -0.020898321643471718, -0.020596304908394814, 0.009353742003440857, -0.003331671468913555, -0.009491903707385063, 0.030117977410554886, 0.01908210851252079, 0.012690035626292229, -0.017078785225749016, -0.0016324761090800166, 0.12997251749038696, 0.030969230458140373, -0.025823738425970078, 0.0278230682015419, 0.025159770622849464, -0.0808122381567955, -0.0036173474509269, -0.008902025409042835, 0.003404824063181877, -0.0230092890560627, -0.03404529020190239, 0.024598615244030952, 0.013545555993914604, -0.025439025834202766, 0.011951087042689323, -0.020540112629532814, -0.010077380575239658, 0.020575348287820816, 0.043889813125133514, 0.014394641853868961, 0.01827182061970234, 1.71

In [12]:
# init elastic search
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': 'bdf182f03357', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'x9jFL0_oTOSH3X-ZpWGj6Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text-vector": {"type": "dense_vector", "dims": 768, "index":True, "similarity": "cosine" }  # potrebnao je definisati dimenstion dim , i similarity matrix ima ih nekoliko
        }
    }
}

index_name = "course-questions1"

es_client.indices.delete(index=index_name, ignore_unavailable=True) # (brisemo indek ako postoji ako ne onda kreiramo) ova linija nam omogucava to da nemoramo stalno da menjamo index_name kao  u prethodinm slucajevima
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions1'})

In [14]:
# sada  indeksiramo dokument koji sadrzi embedings
for doc in operations:
    try:
        es_client.index(index=index_name,document=doc)
    except Exception as e:
        print(e)

In [15]:
serach_term = "windows or mac?"
vector_serach_term = model.encode(serach_term) # svaki serach_term mora da se prebaci u vector

In [16]:
query ={
    "field":"text-vector",
    "query_vector" : vector_serach_term,
    "k" : 5,
    "num_candidates" : 10000 # this number can be also different need to check
}

In [17]:
res = es_client.search(index=index_name, knn=query, source=["text","section","question","course"])
res['hits']['hits']


[{'_index': 'course-questions1',
  '_id': 'W2EoapEB0OWfAG-1JUg4',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions1',
  '_id': 'bmEoapEB0OWfAG-1cEtj',
  '_score': 0.61347336,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares

In [22]:
response = es_client.search(
    index=index_name,
    query={
        "bool":{
            "must":{
                "multi_match": {
                    "query":"windows or python",
                    "fields":["text","question","course","title"],
                    "type":"best_fields"
                }
            },
            "filter":{
                "term":{
                    "course":"data-engineering-zoomcamp"
                }
            }
        }
    }
)

In [30]:
response['hits']['hits']


[{'_index': 'course-questions1',
  '_id': 'W2EoapEB0OWfAG-1JUg4',
  '_score': 1.4937059,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text-vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,
    -0.015976183116436005,
    0.008629541844129562,
    -0.0244789905846

In [28]:
knn_query = {
    "field":"text-vector",
    "query_vector" : vector_serach_term,
    "k" : 5,
    "num_candidates" : 10000 # this number can be also different need to check
}
    
response = es_client.search(
    index=index_name,
    query={
            "match":{
               "course":"data-engineering-zoomcamp"
            },
        },
    knn = knn_query,
    size = 5
           
    
)

In [29]:
response['hits']['hits']

[{'_index': 'course-questions1',
  '_id': 'W2EoapEB0OWfAG-1JUg4',
  '_score': 1.4937059,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text-vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,
    -0.015976183116436005,
    0.008629541844129562,
    -0.0244789905846